# Preprocessing transcripts

In [1]:
#%pip install pandarallel

In [2]:
# Imports
import pickle
import pandas as pd
import numpy as np
import re
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
# Load transcripts from pickle
raw_transcripts = (
    pd.read_pickle('data/clean/subset_party_imputed_v3_2015_version.p')
        .query('source == "parliament"')
        .drop(columns = ['index'])
    )  # TODO: change dataset after retrieval pipeline is cleaned up

In [4]:
print(raw_transcripts.shape)
raw_transcripts.head()

(159723, 6)


,doc,source,start_time,full_name,level,party
12,Tak. Danmarks kolonihistorie handler om militæ...,parliament,2019-10-03,Aki-Matilda Høegh-Dam,NaN,SIU
13,"Jeg mener, at det er personen, der kan snakke ...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,NaN,SIU
14,"Det handler jo ikke om, hvad jeg synes. Det ha...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,NaN,SIU
15,"Jeg ved ikke, om det – når man snakker om at b...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,NaN,SIU
16,"Det er ikke noget, jeg bare mener; det er noge...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,NaN,SIU


In [5]:
raw_transcripts['start_time'].sort_values()

42484     2015-07-02
42483     2015-07-02
42482     2015-07-02
146643    2015-07-03
235398    2015-07-03
             ...    
234429    2022-10-06
234428    2022-10-06
234427    2022-10-06
234443    2022-10-06
475590    2022-10-06
Name: start_time, Length: 159723, dtype: object

In [6]:
query_string = 'stærkt mandat'

[doc for doc in raw_transcripts['doc'] if query_string in doc][:30]

['Jeg synes, at statsministeren meget klart sagde fra over for et forslag, som lå helt uden for, hvad man kan acceptere fra dansk side, og jeg er også glad for, at når vi lægger tonediskussionen lidt til side, lyder det, som om vi stadig væk har Venstre på en budgetrestriktiv linje, og at vi kan blive enige om det, sådan at ministeren rejser til Bruxelles og de her vigtige forhandlinger med et stærkt mandat i ryggen.',
 'Tak. Så det blev simpelt hen udløst af statsministerens spørgsmål om, hvor finansieringen skulle komme fra. Jeg synes ikke helt, det var et svar på mit spørgsmål om, hvorfor Venstre brød med en 30 års tradition. For er det grundlæggende ikke vigtigt, at vi giver regeringen et stærkt mandat i ryggen, når regeringen skal ud at forhandle på vegne af Danmark og danske interesser, hvilket jeg ved at Venstre også er interesseret i?',
 'Jamen det er svaret. Altså, svaret er, at statsministeren har nægtet, finansministeren har nægtet, udenrigsministeren har nægtet at svare på,

In [7]:
def remove_phrases(doc, harshness = 'low'):
    """
    Remove phrases from a lowercased document.
    """
    
    if harshness == 'high':
        # Remove names – requires uppercase and is thus done separately!
        name_pattern = r'(?:[Hh]r\. |[Ff]ru |[Ff]røken )[A-ZÆØÅ][a-zæøå]+(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)? '
        doc = re.sub(name_pattern, '', doc)

        # Remove party names
        party_pattern = r'Socialdemokrat[ietsrnes]*|Venstre[s]*|Dansk Folkeparti[s]*|Enhedslisten[s]*|SF[s]*|Konservative[s]*|Radikale Venstre[s]*|De Radikale[s]*|Radikale[s]*|Nye Borgerlige[s]*|Liberal Alliance[s]*|Alternativet[s]*|Frie Grønne[s]*'
        doc = re.sub(party_pattern, '', doc)

    doc = doc.lower()

    # Remove procedural thank yous of different kinds
    politeness_pattern = r'^[\w\s,]*tak[\w\s,]*. |^tak'
    doc = re.sub(politeness_pattern, '', doc)

    # remove superfluous whitespace
    doc = re.sub(r'\s+', ' ', doc)
    
    return doc


In [10]:
test_doc = 'Tak, fru formand. Spørgsmålet fra SFs formand er ret enkelt. Er fru Pernille Vermund enig i at indføre brugerbetaling? Det er vel De Radikales politik?'

In [11]:
remove_phrases(test_doc, harshness='high')

'spørgsmålet fra formand er ret enkelt. er enig i at indføre brugerbetaling? det er vel politik?'

In [58]:
import re
import string
import nltk
for dependency in ['punkt', 'wordnet', 'omw-1.4', 'stopwords', 'averaged_perceptron_tagger']:
    nltk.download(dependency)

def preproc_docs(text, harshness = 'low'):
    #Lowercasing words
    text = text.lower()
    
    #Removing HTML tag
    text = re.sub(r'&amp', '', text)

    #Replace "&" with "and"
    text = re.sub(r'&','and', text)
    
    #Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation.replace('-',''))) #Taking hyphens out of punctuation to remove
    text = re.sub(r' - ','', text) #removing dash lines bounded by whitespace (and therefore not part of a word)
    text = re.sub(r'…', '', text)
    text = re.sub(r'[â€˜â€™â€œâ€â€”]','',text) #removing punctuation that is not captured by string.punctuation

    #Removing numbers
    text = re.sub(r'[0-9.]','', text)

    # Removing idiosynchratic characters in our data
    text = re.sub(r'-\n|\n-|\na-|\nb-|â€“|Â«|--|’', '', text)
    text = re.sub(r'- ', ' ', text)

    #Removing separators and superfluous whitespace
    text = text.strip()
    text = re.sub(r' +',' ',text)

    return text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mathiasbruun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [65]:
# function to get number of words in a string
def get_word_count(text):
    return len(text.split())

def preproc_pipeline(raw_transcripts, harshness = 'low'):

    # Minimal preproc: lowercase, remove punctuation, remove numbers, remove separators, remove superfluous whitespace
    if harshness == 'low':
        raw_transcripts['doc'] = raw_transcripts['doc'].parallel_apply(preproc_docs)

    # Moderate preproc: minimal preproc + remove thank yous/procedural fluff + remove very short docs
    if harshness == 'moderate':
        raw_transcripts['doc'] = raw_transcripts['doc'].parallel_apply(lambda x: remove_phrases(x, harshness='low'))
        raw_transcripts['doc'] = raw_transcripts['doc'].parallel_apply(lambda x: preproc_docs(x))

        raw_transcripts['word_count'] = raw_transcripts['doc'].parallel_apply(lambda x: get_word_count(x))
        raw_transcripts = raw_transcripts.loc[raw_transcripts.word_count > 10].reset_index(drop=True)

    # Harsh preproc: moderate preproc + remove names, remove party names
    if harshness == 'high':
        raw_transcripts['doc'] = raw_transcripts['doc'].parallel_apply(lambda x: remove_phrases(x, harshness='high'))
        raw_transcripts['doc'] = raw_transcripts['doc'].parallel_apply(lambda x: preproc_docs(x))

        raw_transcripts['word_count'] = raw_transcripts['doc'].parallel_apply(lambda x: get_word_count(x))
        raw_transcripts = raw_transcripts.loc[raw_transcripts.word_count > 10].reset_index(drop=True)

    return raw_transcripts

In [66]:
# Minimal preproc
transcripts_low = preproc_pipeline(raw_transcripts, harshness = 'low')

# Moderate preproc
transcripts_moderate = preproc_pipeline(raw_transcripts, harshness = 'moderate')

# Harsh preproc
transcripts_high = preproc_pipeline(raw_transcripts, harshness = 'high')

In [83]:
transcripts_high.doc.head(50)

len(transcripts_moderate)

132913

In [73]:
# increase pandas print length 
pd.set_option('display.max_colwidth', 100)

In [84]:
transcripts_low.to_pickle('data/clean/preprocessed_docs_2015_low.p')
transcripts_moderate.to_pickle('data/clean/preprocessed_docs_2015_moderate.p')
transcripts_high.to_pickle('data/clean/preprocessed_docs_2015_high.p')

In [95]:
[t for t in transcripts_low.doc if 'pølsesnak' in t][:10]

['pølsesnak',
 '-timersreglen hvor er de billige boliger som disse mennesker skal bo i når de ikke længere har råd til at bo der hvor de bor nu alle spørgsmål bliver besvaret med endeløs pølsesnak om at det skal kunne betale sig at arbejde og at hvis det kan det kommer der også flere i arbejde men det tror dansk folkeparti og deres kollegaer jo ikke engang selv på mennesker bliver fattigere efter regeringens egne tal men man forventer kun at kommer i arbejde – måske det viser jo bare at det ikke er det det drejer sig om den virkelige dagsorden er en anden det drejer sig om at skaffe råderum til skattelettelser det behøver vi ikke at diskutere det fremgår af forligsteksten at hr kristian thulesen dahl og går rundt og siger noget andet er der jo ikke så meget at gøre ved det må de selv rydde op i men dagsordenen er jo også at trykke mindstelønnen alle økonomer uanset politisk observans er enige om at der er en sammenhæng mellem de laveste sociale ydelser og mindstelønnen når de sociale y